In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import warnings


# Testes de Hipótese

## Entendendo valores-p

Suponhamos que você tenha um website com um banner de propaganda, e que 20% das pessoas que visitam o site clicam neste banner. A area de comunicação desenvolveu um novo banner e, ao implantar este novo banner, dos primeiros 30 clientes que o viram, 
11 pessoas clicaram no banner novo.

Como podemos descobrir se isso é um efeito aleatório ou um impacto real? Vamos simular este efeito usando amostragem computacional:

In [ ]:
p = 11/30
p

In [ ]:
primeiros_30 = np.random.choice([0, 1], p=[1 - p, p], size=30)
np.sum(primeiros_30)


In [ ]:
np.mean(primeiros_30)

Estamos utilizando a proporção observada nos 30 primeiros clientes que viram o banner para gerar amostrar aleatórias. O array `primeiros_30` contém uma amostra de tamanho 30, onde cada elemento tem **chance p** de ser 1 e **chance 1-p** de ser 0.

Agora, vamos simular 1000 amostras de tamanho 30 com a nossa **% de cliques original (20%)**:

In [ ]:
dist_pop = []
for i in range(0, 100000):
    amostra = np.random.choice([0, 1], p=[0.8, 0.2], size=30)
    dist_pop.append(np.mean(amostra))


O vetor que criamos acima tem é a **distribuição de amostragem de p**:

In [ ]:
dist_pop = np.array(dist_pop)
sns.histplot(dist_pop)


Agora conseguimos calcular, a partir da nossa distribuição simulada, **a probabilidade de observarmos uma amostra com 11 acessos em 30 impressões**:

In [ ]:
len(dist_pop[dist_pop >= p]) / 100000


Se a taxa real era de 20%, a chance de observarmos 11 cliques em 30 acessos (uma taxa de 36,67%) seria de 2,6%

## Utilizando um teste-t simples

O teste-t simples nos permite comparar a média de uma amostra contra a média de um população e calcular a probabilidade da amostra pertencer à população:

* __Null hypothesis (H0): u1 = u2, a média da amostra é a média da população.__
* __Alternative hypothesis (H1): u1 != u2, a média da amostra é diferente da média da população.__

Para avaliar o resultado de um teste-t vamos utilizar o conceito de **valor-p** e **nível de confiabilidade**:

**Se nosso *valor-p* for inferior à *(1 - Nível de Confiabilidade)* vamos REJEITAR A HIPÓTESE NULA**

Vamos utilizar um teste-t para compara valores de calorias medidos em uma barra energética contra o valor afirmado na embalagem:

In [ ]:
from scipy import stats

In [ ]:
amostr_barra_energetica = [
    20.70, 27.46,22.15,19.85,21.29,24.75,
    20.75,22.91,25.34,20.33,21.54,
    21.08,22.14,19.56,21.10,18.04,
    24.12,19.95,19.72,18.28,16.26,
    17.46,20.53,22.12,25.06,22.44,
    19.08,19.88,21.39,22.33,25.79
]

valor_rotulo = 20

stats.ttest_1samp(amostr_barra_energetica, valor_rotulo)


In [ ]:
np.mean(amostr_barra_energetica)

In [ ]:
np.std(amostr_barra_energetica)

# Paired T-Test

https://raw.githubusercontent.com/Opensourcefordatascience/Data-sets/master/blood_pressure.csv

Vamos utilizar o dataset e utilizar um teste-t em par para medir se um tratamento para hipertensão foi efetivo ou não:


In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/Opensourcefordatascience/Data-sets/master/blood_pressure.csv"
)


In [ ]:
df.info()


In [ ]:
df

## Assumption check 

- [ ]  The samples are independently and randomly drawn

**AND**

- [ ]  The distribution of the residuals between the two groups should follow the normal distribution

**AND**

- [ ]  The variances between the two groups are equal

**OR**

- [ ]  Approximately equal sample sizes

https://www.youtube.com/watch?v=OyB_w4XNQ58&t=315s

### Checking Normal distribution by Q-Q plot graph
https://www.statisticshowto.datasciencecentral.com/assumption-of-normality-test/

In [ ]:
df["diff_"] = df["bp_before"] - df["bp_after"]


In [ ]:
sns.displot(df["diff_"])


In [ ]:
plt.figure(figsize=(10, 10))
stats.probplot(df["diff_"], plot=plt)
plt.show()


**Note:-** The corresponding points lies very close to line that means are our sample data sets are normally distributed

### Checking Normal distribution by method of `Shapiro stats`
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html

In [ ]:
stats.shapiro(df["diff_"])


### Checking if the variance is equal between samples

In [ ]:
df["bp_after"].std()


In [ ]:
df["bp_before"].std()


In [ ]:
stats.levene(df.bp_after, df.bp_before)


O teste-t é robusto à violações das premissas - mais robustos que os testes de normalidade e igualdade de variância. Portanto, embora devamos tomar cuidado com a utilização em casos extremos (especialmente com amostras pequenas), desvios das premissas do teste são toleráveis.

# Aplicando o teste de Hipótese

Testando a efetividade do tratamento.

In [ ]:
df[["bp_before", "bp_after"]].describe()


In [ ]:
df[["bp_before", "bp_after"]].plot(kind="hist", alpha=0.7)


## Qual nossa hipótese nula?

* __Null hypothesis (H0): u1 = u2, a média da amostra antes do tratamento é igual a média da amostra após tratamento.__
* __Alternative hypothesis (H1): u1 != u2, a média da amostra antes do tratamento é diferente da média da amostra após tratamento__

In [ ]:
stats.ttest_rel(df['bp_before'], df['bp_after'])


In [ ]:
df['bp_before'] - df['bp_after']

In [ ]:
stats.ttest_1samp(df['bp_before'] - df['bp_after'], 0)


Como o valor p está abaixo de 0,05 podemos rejeitar a hipótese nula, ou seja, que pressão arterial antes da intervenção tem a mesma média que a pressão arterial depois da intervenção!

# Two Sample T-Test

In [ ]:
tb_bike = pd.read_csv("data/tb_bikesharing.csv")


In [ ]:
tb_bike.head()


In [ ]:
sns.kdeplot(data=tb_bike, x="cnt", hue="workingday", common_norm=False)


In [ ]:
sns.boxplot(data=tb_bike, y="cnt", x="workingday")


In [ ]:
tb_bike_fds = tb_bike[tb_bike["workingday"] == 0].copy()
tb_bike_sem = tb_bike[tb_bike["workingday"] == 1].copy()


In [ ]:
tb_bike_sem['cnt']

### Assumption check 

- [ ]  The samples are independently and randomly drawn

**AND**

- [ ]  The distribution of the residuals between the two groups should follow the normal distribution

**AND**

- [ ]  The variances between the two groups are equal

**OR**

- [ ]  Approximately equal sample sizes

#### Normal?

In [ ]:
plt.figure(figsize=(10, 10))
stats.probplot(tb_bike_fds["cnt"], plot=plt)
plt.show()


In [ ]:
plt.figure(figsize=(10, 10))
stats.probplot(tb_bike_sem["cnt"], plot=plt)
plt.show()


#### Equal variance or equal sample size?

In [ ]:
tb_bike.groupby("workingday")["cnt"].var()


In [ ]:
stats.levene(tb_bike_fds["cnt"], tb_bike_sem["cnt"])


In [ ]:
tb_bike["workingday"].value_counts()


Os grupos estão desbalanceados e a variância é diferente! Será que podemos utilizar um teste-t? Nas palavras de Richard Nixon: <i>"We could do it... but it would be wrong"</i>. Como temos amostras grandes (n>>30) podemos rebalancear os dados!

<i>Ressalva: se as amostras tem uma proporção de 1,5:1 (300:200 pontos, por exemplo), podemos proceder sem grandes problemas.</i>

In [ ]:
tb_bike_sem_smp = tb_bike_sem.sample(231, random_state=42).reset_index(drop=True)


In [ ]:
tb_bike_sem_smp.shape


In [ ]:
stats.ttest_ind(tb_bike_fds["cnt"], tb_bike_sem_smp["cnt"])


# One tailed-side t-test

A hipótese nula padrão no teste-t é que a média das duas amostra é igual. Rejeitar esta hipótese significa afirmar que a média de uma amostra é maior ou menor que a média da outra amostra. No entanto, muitas vezes nos interessa determinar o desvio da média da amostra apenas em um sentido. Neste podemos utilizar o teste-t unilateral - one a hipótese nula **H0** é que **A <= B** OU  **A >= B**.

Alguns exemplos práticos: uma indústria de fertilizantes quer saber se seu novo produto aumenta a produtividade por hectar de milho plantado, um agência de comunicação deseja saber se um novo banner aumenta a taxa de cliques em um website, um cientista de dados deseja saber se seu novo sistema de recomendação aumenta as vendas.

In [ ]:
tb_bike["temp_classif"] = np.where(tb_bike["temp"] > np.mean(tb_bike["temp"]), 1, 0)


In [ ]:
sns.boxplot(data=tb_bike, x="temp_classif", y="cnt")


In [ ]:
tb_bike_quente = tb_bike[tb_bike["temp_classif"] == 1].copy()
tb_bike_frio = tb_bike[tb_bike["temp_classif"] == 0].copy()


In [ ]:
tb_bike_frio.shape


In [ ]:
tb_bike_quente.shape


In [ ]:
stats.ttest_ind(tb_bike_frio["cnt"], tb_bike_quente["cnt"], alternative="greater")


In [ ]:
stats.ttest_ind(tb_bike_frio["cnt"], tb_bike_quente["cnt"])


# ANOVA

E se quisermos comparar a média de mais de um grupo? O método ANOVA testa a hipótese nula onde todas as amostras foram feitas da mesma população, ou seja, que a média dos diferentes grupos é identica. Rejeitar a hipótese nula significa dizer que **pelo menos um grupo não é uma amostra da mesma população, ou seja, pelo menos a média de um grupo é diferente dos grupos restantes**

In [ ]:
tb_bike["estacao"] = tb_bike["season"].map(
    {1: "Inverno", 2: "Primavera", 3: "Verão", 4: "Outono"}
)


In [ ]:
sns.boxplot(data=tb_bike, x="estacao", y="cnt")


In [ ]:
tb_bike_season = [tb_bike[tb_bike["season"] == i]["cnt"] for i in range(1, 5)]


In [ ]:
lista_vetores = []
for i in range(1,5):
    lista_vetores.append(tb_bike[tb_bike["season"] == i]["cnt"])


In [ ]:
lista_vetores[0]

In [ ]:
tb_bike_season = [tb_bike[tb_bike["season"] == i]["cnt"] for i in range(1, 5)]

In [ ]:
tb_bike_season[3]

In [ ]:
stats.f_oneway(
    tb_bike_season[0], tb_bike_season[1], tb_bike_season[2], tb_bike_season[3]
)


E se quisermos saber a proporção entre os grupos?

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd


In [ ]:
nivel_confiabilidade = 0.95
tukey_fit = pairwise_tukeyhsd(
    endog=tb_bike["cnt"], groups=tb_bike["estacao"], alpha=(1-nivel_confiabilidade)
)


In [ ]:
tukey_fit.summary()
